# Initilization


In [1]:
import pickle 
import torch
from torch_geometric.data import Data
from ExplanationEvaluation.models.model_selector import model_selector
from ExplanationEvaluation.configs.selector import Selector
from ExplanationEvaluation.datasets.dataset_loaders import load_dataset
from ExplanationEvaluation.tasks.replication import get_classification_task, to_torch_graph


dataset_name = "bbbp"

# Load pretrained models
_dataset = "bbbp"
_explainer = "actexplainer"

_folder = 'replication' # One of: replication, extension

config_path = f"GNN-explain/codebase/ExplanationEvaluation/configs/{_folder}/explainers/{_explainer}/{_dataset}.json"
radius = 3
config = Selector(config_path)
config = config.args.explainer
model, checkpoint = model_selector(config.model,
                                    config.dataset,
                                    pretrained=True,
                                    return_checkpoint=True)
if config.eval_enabled:
    model.eval()
    
graphs, features, labels, train_mask, _, test_mask = load_dataset(config.dataset)
task = get_classification_task(graphs)

graphs = [g for i, g in enumerate(graphs) if train_mask[i]]
features = [f for i, f in enumerate(features) if train_mask[i]]
labels = [l for i, l in enumerate(labels) if train_mask[i]]


features = torch.tensor(features)
labels = torch.tensor(labels)
graphs = to_torch_graph(graphs, task)
print(features.shape, labels.shape, len(graphs))
dataset = [Data(x=features[i], edge_index=graphs[i], y=labels[i]) for i in range(len(graphs))]

    
    
    

This model obtained: Train Acc: 0.8811, Val Acc: 0.8415, Test Acc: 0.8232.
Loading BBBP dataset
torch.Size([1312, 133, 13]) torch.Size([1312, 2]) 1312


/tmp/ipykernel_1378602/145080924.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  features = torch.tensor(features)


# ESPAM


In [8]:
def relable_nodes(feature, edge_index, node_subset):
    node_subset = torch.unique(node_subset, return_inverse=True, sorted=True)
    node_dict = {node_subset[0][i].item(): node_subset[1][i].item() for i in range(node_subset[0].shape[0])}
    edge_index = torch.tensor([[node_dict[u.item()], node_dict[v.item()]] for u, v in edge_index.T]).T
    return feature[node_subset[0]], edge_index

In [7]:
import dgl
from torch_geometric.utils import k_hop_subgraph, mask_to_index, index_to_mask
import torch_geometric.transforms as T

def features_to_graph(data,z,l):
    center_nodes = []
    for i in range(len(z)):
        if(z[i] == 0):
            center_nodes.append(i)
           
    kh_subset, kh_edge_index, kh_mapping, kh_edge_mask = k_hop_subgraph1(center_nodes, l, data.edge_index, 
                                                                                relabel_nodes=False)
    g = dgl.graph((data.edge_index[0], data.edge_index[1]))
    #print(type(g))
    #hub_ego = nx.ego_graph(g, center_nodes[0])
    #print(type(hub_ego), type(g))
    ''''
    #print('nodes',kh_subset)
    #remove edges
    remove_edges = []
    for j in range(len(center_nodes)):
        for i in range(len(data.edge_index[0])):
            if((data.edge_index[0][i] == center_nodes[j]) or (data.edge_index[1][i] == center_nodes[j])):
                remove_edges.append(i)
    #end remove edges
    #print(remove_edges)
    #to_be_remove = torch.logical_and(to_be_remove,torch.logical_not(kh_edge_mask))
    #print('tbr',to_be_remove)
    #if(r):
    g.remove_edges(remove_edges) ###mask_to_index(kh_edge_mask))
    '''
    
    g.remove_edges(mask_to_index(kh_edge_mask))
    
    
    edges_src, edges_dst = g.edges()
    #print(data.num_nodes, data.num_edges)
    edge_index = torch.tensor([edges_src.numpy(), edges_dst.numpy()], dtype=torch.long)
    data = Data(x=data.x, edge_index=edge_index)#, y=data.y, shape=data.shape)
    
    return data

In [6]:
# Khop


from typing import List, Optional, Tuple, Union

import torch
from torch import Tensor

from torch_geometric.typing import OptTensor, PairTensor

from torch_geometric.utils.mask import index_to_mask
from torch_geometric.utils.num_nodes import maybe_num_nodes

def k_hop_subgraph1(
    node_idx: Union[int, List[int], Tensor],
    num_hops: int,
    edge_index: Tensor,
    relabel_nodes: bool = False,
    num_nodes: Optional[int] = None,
    flow: str = 'source_to_target',
    directed: bool = False,
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    r"""Computes the induced subgraph of :obj:`edge_index` around all nodes in
    :attr:`node_idx` reachable within :math:`k` hops.

    The :attr:`flow` argument denotes the direction of edges for finding
    :math:`k`-hop neighbors. If set to :obj:`"source_to_target"`, then the
    method will find all neighbors that point to the initial set of seed nodes
    in :attr:`node_idx.`
    This mimics the natural flow of message passing in Graph Neural Networks.

    The method returns (1) the nodes involved in the subgraph, (2) the filtered
    :obj:`edge_index` connectivity, (3) the mapping from node indices in
    :obj:`node_idx` to their new location, and (4) the edge mask indicating
    which edges were preserved.

    Args:
        node_idx (int, list, tuple or :obj:`torch.Tensor`): The central seed
            node(s).
        num_hops (int): The number of hops :math:`k`.
        edge_index (LongTensor): The edge indices.
        relabel_nodes (bool, optional): If set to :obj:`True`, the resulting
            :obj:`edge_index` will be relabeled to hold consecutive indices
            starting from zero. (default: :obj:`False`)
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)
        flow (str, optional): The flow direction of :math:`k`-hop aggregation
            (:obj:`"source_to_target"` or :obj:`"target_to_source"`).
            (default: :obj:`"source_to_target"`)
        directed (bool, optional): If set to :obj:`False`, will include all
            edges between all sampled nodes. (default: :obj:`True`)

    :rtype: (:class:`LongTensor`, :class:`LongTensor`, :class:`LongTensor`,
             :class:`BoolTensor`)

    Examples:

        >>> edge_index = torch.tensor([[0, 1, 2, 3, 4, 5],
        ...                            [2, 2, 4, 4, 6, 6]])

        >>> # Center node 6, 2-hops
        >>> subset, edge_index, mapping, edge_mask = k_hop_subgraph(
        ...     6, 2, edge_index, relabel_nodes=True)
        >>> subset
        tensor([2, 3, 4, 5, 6])
        >>> edge_index
        tensor([[0, 1, 2, 3],
                [2, 2, 4, 4]])
        >>> mapping
        tensor([4])
        >>> edge_mask
        tensor([False, False,  True,  True,  True,  True])
        >>> subset[mapping]
        tensor([6])

        >>> edge_index = torch.tensor([[1, 2, 4, 5],
        ...                            [0, 1, 5, 6]])
        >>> (subset, edge_index,
        ...  mapping, edge_mask) = k_hop_subgraph([0, 6], 2,
        ...                                       edge_index,
        ...                                       relabel_nodes=True)
        >>> subset
        tensor([0, 1, 2, 4, 5, 6])
        >>> edge_index
        tensor([[1, 2, 3, 4],
                [0, 1, 4, 5]])
        >>> mapping
        tensor([0, 5])
        >>> edge_mask
        tensor([True, True, True, True])
        >>> subset[mapping]
        tensor([0, 6])
    """
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    assert flow in ['source_to_target', 'target_to_source']
    if flow == 'target_to_source':
        row, col = edge_index
    else:
        col, row = edge_index

    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)
     
    #print("edge")
    #print(col)
    #print(row)
    
    if isinstance(node_idx, (int, list, tuple)):
        node_idx = torch.tensor([node_idx], device=row.device).flatten()
    else:
        node_idx = node_idx.to(row.device)

    subsets = [node_idx]
    tmp_subset = []
    for i in range(num_hops):
        node_mask.fill_(False)
        try:
            node_mask[subsets[-1]] = True
        except:
            print(node_mask)
            print(len(node_mask))
            print(subsets)
            print(subsets[-1])
            raise
        torch.index_select(node_mask, 0, row, out=edge_mask)
        if(i < num_hops-1): ### j'ai ajoute ce test pour enlever la derniere couronne de noeuds
            subsets.append(col[edge_mask])
        else:
            tmp_subset.append(col[edge_mask])

    subset, inv = torch.cat(subsets).unique(return_inverse=True, sorted=False)
    #print("moins", subset)
    node_mask.fill_(False)
    node_mask[subset] = True
    #print("nmsk",node_mask)
    
    #print("emsk",edge_mask)
    if not directed:
        #print("row")
        #print(node_mask[row])
        #print(node_mask[col])
        edge_mask = node_mask[row] | node_mask[col] ## j'ai change le & en |
        #il suffit qu'un noeud adjacent ait ete atteind pour qu'on enleve l'arete (et non pas les deux comme avant)
    #print("emsk",edge_mask)
    
    edge_index = edge_index[:, edge_mask]
    #print("eidx")
    #print(edge_index)
    #print(inv)
    #print(edge_mask)
    subsets.append(tmp_subset[0])
    subset, inv = torch.cat(subsets).unique(return_inverse=True, sorted=False)
    inv = inv[:node_idx.numel()]
    node_mask[subset] = True
    
    if relabel_nodes:
        node_idx = row.new_full((num_nodes, ), -1)
        node_idx[subset] = torch.arange(subset.size(0), device=row.device)
        edge_index = node_idx[edge_index]
    return subset, edge_index, inv, edge_mask

## ESPAM Function

In [29]:
import networkx as nx
from torch_geometric.utils import to_networkx
def espam_ego_networks_average(data, model, class_index, ll=4):
    number_of_nodes = data.x.shape[0]
    G = to_networkx(data)
    isolated_nodes = list(nx.isolates(G))
    ego_sets = []
    model_score = lambda data: model(data.x, data.edge_index)
    graph_score = model_score(data).detach().softmax(dim=1)[0][class_index]
    ego_contributions = []
    complement_ego_sets = []
    for l in range(1, ll):
        ego_sets.append([k_hop_subgraph1(node_idx, l, data.edge_index) for node_idx in range(number_of_nodes) if node_idx not in isolated_nodes])
        while ego_sets[-1][-1][1].shape[1] == 0:
            print("!")
            ego_sets[-1].pop()
            number_of_nodes -= 1
        ego_sets[-1] = list(map(lambda x: list(relable_nodes(data.x, x[1], x[0])) + [x[2], x[3]], ego_sets[-1]))
        complement_ego_sets.append(
            [features_to_graph(data, torch.ones(data.x.shape[0]).scatter_(-1, torch.tensor([node_idx]), 0), l) for
             node_idx in range(number_of_nodes) if node_idx not in isolated_nodes])
        complement_scores = list(
            map(lambda x: model_score(x).detach().softmax(dim=1)[0][class_index], complement_ego_sets[-1]))
        complement_sums = sum(complement_scores)
        try:
            ego_scores = list(
            map(lambda x: model_score(Data(x=x[0], edge_index=x[1])).detach().softmax(dim=1)[0][class_index], ego_sets[-1]))
        except:
            print(list(enumerate(map(lambda x: x[1].shape, ego_sets[-1]))))
            print(l)

            raise
        ego_sums = sum(ego_scores)
        w = (graph_score + complement_sums) / (complement_sums + ego_sums)
        ego_contributions.append(list(
            map(lambda node_index: w * ego_scores[node_index] + (1 - w) * (-complement_scores[node_index]) ,
                range(number_of_nodes- len(isolated_nodes)))))
    node_contributions = torch.tensor(ego_contributions).sum(dim=0) / (ll-1)
    node_contributions = node_contributions.tolist()
    for index in isolated_nodes:
        node_contributions.insert(index,0)
    return torch.tensor(node_contributions)
    
    
    

## Run ESPAM

In [49]:
## l = 5 for BA2 and l = 2 for others has the best results

from tqdm import tqdm
import pickle 
cnt = 0
for l in range(2, 7):
    espam_values = []
    for data in tqdm(dataset):
        x, edge_index = data.x, data.edge_index
        class_index = model(x, edge_index)[0].argmax()
        try:
            espam_values.append(espam_ego_networks_average(data, model, class_index, l))
        except:
            print("error")
            cnt += 1
            if cnt < 3:
                continue
            raise
    with open(f"{dataset_name}_ESPAM_results_{l=}.pkl", "wb") as file:
        pickle.dump(espam_values, file)

100%|██████████| 800/800 [05:20<00:00,  2.49it/s]


# Baselines

## GNN-Explainer, GradCAM, PGExplainer, and PGM Explainer

In [2]:
from torch_geometric.explain import PGExplainer, Explainer, GNNExplainer
from tqdm import tqdm
from graphxai.explainers import GradCAM
from ExplanationEvaluation.explainers.PGMExplainer import PGMExplainer
from ExplanationEvaluation.explainers.PGExplainer import PGExplainer
import warnings
warnings.filterwarnings("ignore")

explain_dict = {}
explainer_dict = {"PGExplainer":PGExplainer(model, graphs, features, task),
    "GNNExplainer": Explainer(
        model=model,
        algorithm=GNNExplainer(),
        explanation_type='model',
        node_mask_type='object',
        edge_mask_type='object',
        model_config=dict(
            mode='multiclass_classification',
            task_level='graph',
            return_type='raw',)
            ),
    "GradCAM": GradCAM(model), 
    "PGM": PGMExplainer(model, graphs, features, task, policy_name=None)
}

explainer_dict["PGExplainer"].prepare(torch.where(torch.tensor(train_mask))[0])

for i in tqdm(range(len(graphs))):
    if not train_mask[i]:
        continue
    explain_dict[i] = {}
    for explainer_name, explainer in explainer_dict.items():
        if explainer_name == "GNNExplainer":
            explain_dict[i][explainer_name] = explainer(x=features[i], edge_index=graphs[i])
        elif explainer_name == "GradCAM":
            explain_dict[i][explainer_name] = explainer.get_explanation_graph(x=features[i], edge_index=graphs[i])
        elif explainer_name in ["PGM", "PGExplainer"]:
            explain_dict[i][explainer_name] = explainer.explain(i)
    

100%|██████████| 2000/2000 [29:38<00:00,  1.12it/s]


## GraphSVX

In [ ]:
import torch
import pickle 
from tqdm import tqdm 
from ExplanationEvaluation.explainers.SVXExplainer import GraphSVX
import numpy as np
import random 
from torch_geometric.data import Data
device ="cpu"
hfid = None
explainer_name = "SVX"
explain_dict = {}
seed = None
#print(explanations.keys())
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    np.random.seed(seed)
    random.seed(seed)
for i in tqdm(range(len(graphs))):
    if not train_mask[i]:
        continue
    explain_dict[i] = {}
    
    x, edge_index = features[i], graphs[i]
    best_hfid = (0, 0,0,0)
    svx_explainer = GraphSVX(model, [edge_index], torch.unsqueeze(x, 0), "graph", gpu=True)
    svx_values = svx_explainer.explain_graphs()
    explain_dict[i]["SVX"] = svx_values
with open(f"{_dataset}_svx_{seed}.pkl", "wb") as file:
    pickle.dump(explain_dict, file) 

# Evaluation


### Metric Functions


In [2]:
def fidelity(data, selected_nodes, model, target_class, remove_mask_function):
    #z = torch.ones(data.x.shape[0]).bool()
    #z[center_indices] = False
    x, edge_index = data.x, data.edge_index
    complement_mask = ~selected_nodes
    masked_graph = remove_mask_function(data, complement_mask)#features_to_graph(data, z, 1)
    return model(x, edge_index).softmax(-1)[0][target_class] - model(masked_graph.x, masked_graph.edge_index).softmax(-1)[0][target_class]


def infidelity(data, selected_nodes, model, target_class, remove_mask_function):
    #z = torch.zeros(data.x.shape[0]).bool()
    #z[center_indices] = True
    x, edge_index = data.x, data.edge_index

    complement_mask = selected_nodes
    masked_graph = remove_mask_function(data, complement_mask)#features_to_graph(data, z, 1)
    return model(x, edge_index).softmax(-1)[0][target_class] - model(masked_graph.x, masked_graph.edge_index).softmax(-1)[0][target_class]


def sparsity(data, selected_nodes, remove_mask_function):
    #data = from_smiles_to_data(smiles)
    #z = torch.zeros(data.x.shape[0]).bool()
    #z[center_indices] = True
    #complement_mask = ~selected_nodes
    #print(complement_mask)

    masked_graph = remove_mask_function(data, selected_nodes)#features_to_graph(data, z, 1)
    return 1 - (masked_graph.true_edges)/(data.edge_index.shape[1])


def hfidelity(data, mask, model, target_class, mask_func):
    number_of_nodes = data.x.shape[0]
    number_of_mask_nodes = mask_func(data, mask).node_mask_size
    fid = fidelity(data, mask, model, target_class, mask_func)
    infid = infidelity(data, mask, model, target_class, mask_func)
    spars = 1 -number_of_mask_nodes/number_of_nodes
    n_fid = fid*spars
    n_infid = infid*(1 - spars)
    hfid = ((1 + n_fid)*(1 - n_infid))/(2+ n_fid - n_infid)
    return hfid

### Mask Functions


In [7]:
from importlib import reload
from torch_geometric.data import Data
from tqdm import tqdm
import numpy as np
from torch_geometric.utils import to_networkx
import networkx as nx

def creat_mask_mol_data(data, mask, is_edge=False, inv=True): 
    mask = mask.to(bool)
    x = data.x.clone()
    edge_index = data.edge_index.clone()
    if not is_edge:
        if inv:
            x[~mask] = torch.zeros_like(x[~mask])
        else:
            x = x[mask]
        edge_mask = mask[edge_index[0]]&mask[edge_index[1]]
        if inv:
            edge_index = edge_index[:, edge_mask]

        else:
            edge_index = edge_index[:, edge_mask]
    else:
        edge_index = edge_index[:, mask]
        edge_mask = mask
        node_mask = torch.zeros(data.x.shape[0])
        node_mask[edge_index[0]] = 1
        node_mask[edge_index[1]] = 1
        x[~node_mask.bool()] = torch.zeros_like(x[~node_mask.bool()])
    res = Data(x=x, edge_index=edge_index, true_edges=sum(edge_mask), node_mask_size =node_mask.sum() if is_edge else mask.sum())
    return res



#####################
#####################
#####################

       

def mask_compute(data, values,  model, target_class, mask_creator_function, min_sparsity):
    number_of_nodes = data.x.shape[0]
    G = to_networkx(data)
    G.remove_edges_from(nx.selfloop_edges(G))
    isolated_nodes = list(nx.isolates(G))
    number_of_nodes -= len(isolated_nodes)
    best_hfid = -100
    fid_make_mask = lambda data, mask: creat_mask_mol_data(data, mask, inv=True)
    for i in range(1,number_of_nodes):
        mask = mask_creator_function(data, (values, number_of_nodes), i)
        fid = fidelity(data, mask, model, target_class, fid_make_mask)
        infid = infidelity(data, mask, model, target_class, fid_make_mask)
        spars = 1 -i/data.x.shape[0]
        if spars < min_sparsity:
            break
        n_fid = fid*spars
        n_infid = infid*(1 - spars)
        hfid = ((1 + n_fid)*(1 - n_infid))/(2+ n_fid - n_infid)
        best_hfid = max(hfid, best_hfid)
    return best_hfid



def top_k_percent_nodes(values, k):
    if len(values) == 2:
        values, num_nodes = values
    else:
        num_nodes = 0
    values = torch.tensor(values)
    number_of_nodes = values.shape[0]
    num_nodes = num_nodes if num_nodes else number_of_nodes
    _, indices = torch.sort(values[:num_nodes], descending=True)
    selected_indices = indices[:k]
    mask = torch.zeros(number_of_nodes)
    mask[selected_indices] = 1
    return mask.to(bool)
    
def add_to_metrics_dict(data, values, mask_creator_function, model, target_class, method):
    best_hfid = mask_compute(data, values, model, target_class, mask_creator_function, 0.5)
    hfidelity_dict[method] += best_hfid
    hfid_val[method].append(best_hfid)

    
    
def devide_metric_values(method, cnt):
    hfidelity_dict[method] /= cnt


hfidelity_dict = {}
hfid_val = {method: [] for method in ["ESPAM"]}
std_dict = {}



'    cnt+= 1\n\nfor method in ["ESPAM"]:\n    devide_metric_values(method, cnt)\n    std_dict[method] = np.std(list(map(lambda x: x.detach().cpu().numpy(), hfid_val[method])))\n    print(f"{l=} {hfidelity_dict[method]} ± {std_dict[method]}")\n    #print(f"protein: {protein}\t espam hfid: {hfidelity_dict[(protein, \'ESPAM\')]} ± {std_dict[(protein, \'ESPAM\')]}\t gstar hfid: {hfidelity_dict[(protein, \'gstarx\')]} ± {std_dict[(protein, \'gstarx\')]}\t shap hfid: {hfidelity_dict[(protein, \'shap\')]} ± {std_dict[(protein, \'shap\')]}")'

## Evaluate ESPAM

In [ ]:
l = 2 if dataset_name != "ba2motifs" else 5
mask_func = lambda smiles, values, k: top_k_percent_nodes(values, k)   
#for l in range(2, 6):
for method in ["ESPAM"]:
    hfidelity_dict[method] = 0
    hfid_val[method] = []
    cnt = 0

with open(f"{dataset_name}_ESPAM_results_{l=}.pkl", "rb") as f:
    ESPAMS = pickle.load(f)
for i, data in tqdm(enumerate(dataset)):
    x, edge_index = data.x, data.edge_index
    #seq = bert_function(seq_id).reshape(1, -1)
    target_class = model(x, edge_index)[0].argmax().item()
    espams = ESPAMS[i]
    add_to_metrics_dict(data, espams, mask_func, model, target_class, "ESPAM")

for method in ["ESPAM"]:
    devide_metric_values(method, cnt)
    std_dict[method] = np.std(list(map(lambda x: x.detach().cpu().numpy(), hfid_val[method])))
    print(f"{l=} {hfidelity_dict[method]} ± {std_dict[method]}")
    #print(f"protein: {protein}\t espam hfid: {hfidelity_dict[(protein, 'ESPAM')]} ± {std_dict[(protein, 'ESPAM')]}\t gstar hfid: {hfidelity_dict[(protein, 'gstarx')]} ± {std_dict[(protein, 'gstarx')]}\t shap hfid: {hfidelity_dict[(protein, 'shap')]} ± {std_dict[(protein, 'shap')]}")"""


In [3]:
def gnn_explainer_mask(data):
    edge_mask = data.edge_mask
    edge_mask = edge_mask > 0.5
    return edge_mask 

def edge_soft_mask(values, number_of_edges):
    values = torch.tensor(values)
    number_of_nodes = values.shape[0]
    _, indices = torch.sort(values, descending=True)
    selected_indices = indices[:number_of_edges]
    mask = torch.zeros(values.shape[0])
    mask[selected_indices] = 1
    return mask.to(bool)

def subgraph_mask(number_of_nodes, indices):
    mask = torch.zeros(number_of_nodes)
    mask[indices] = 1
    mask = mask.bool()
    return mask 


### Run Evaluations

In [ ]:
hfid_dict = {}
min_sparsity = 0.5
hfid_val = {}
hfid_dict = {}
mask_func_edge = lambda x, y: creat_mask_mol_data(x, y, is_edge=True)
mask_func_node = lambda x, y: creat_mask_mol_data(x, y, is_edge=False)

explanation_data = explain_dict#exp1
for index, explainers in tqdm(explanation_data.items()):
    number_of_nodes = features[index].shape[0]
    target_class = model(features[index], graphs[index])[0].argmax().item()
    data = dataset[index].clone()
    for explainer, explanations in explainers.items():
        if explainer not in hfid_dict:
            hfid_dict[explainer] = 0
            hfid_val[explainer] = []
        if explainer == "GNNExplainer":
            mask = gnn_explainer_mask(explanations)
            hfid = hfidelity(data, mask, model, target_class, mask_func_edge)
            hfid_val[explainer].append(hfid)
            hfid_dict[explainer] += hfid
        elif explainer == "PGExplainer":
            values = explanations[1]
            max_h_fid = 0 
            for i in range(values.shape[0]):
                mask = edge_soft_mask(values, i)
                hfid = hfidelity(data, mask, model, target_class, mask_func_edge)
                max_h_fid = max(hfid, max_h_fid)
                sparsity = 1 - mask_func_edge(data, mask).node_mask_size/data.x.shape[0]
                if sparsity < min_sparsity:
                    break
            hfid_val[explainer].append(max_h_fid)  
            hfid_dict[explainer] += max_h_fid
                    
        elif explainer == "PGM":
            mask = explanations[1].bool()
            hfid = hfidelity(data, mask, model, target_class, mask_func_edge)
            hfid_val[explainer].append(hfid)
            hfid_dict[explainer] += hfid
        elif explainer == "gstar":
            explanations = torch.tensor(explanations)
            max_h_fid = 0
            for i in range(number_of_nodes):
                mask = top_k_percent_nodes(explanations, i)
                hfid = hfidelity(data, mask, model, target_class, mask_func_node)
                max_h_fid = max(hfid, max_h_fid)
                sparsity =  1 - i/number_of_nodes
                if sparsity < min_sparsity:
                    break
            hfid_val[explainer].append(max_h_fid)
            hfid_dict[explainer] += max_h_fid
        elif explainer == "SVX":
            values = explanations[0][1][:number_of_nodes]
            max_h_fid = 0
            if values.shape[0] < number_of_nodes:
                values = torch.cat([values, torch.zeros(number_of_nodes - values.shape[0])])
            for i in range(number_of_nodes):
                mask = top_k_percent_nodes(values, i)
                hfid = hfidelity(data, mask, model, target_class, mask_func_node)
                max_h_fid = max(hfid, max_h_fid)
                sparsity =  1 - i/number_of_nodes
                if sparsity < min_sparsity:
                    break
            hfid_val[explainer].append(max_h_fid)
            hfid_dict[explainer] += max_h_fid

        elif explainer == "subgraph":
            mask = subgraph_mask(number_of_nodes, explanations)
            hfid = hfidelity(data, mask, model, target_class, mask_func_node)
            hfid_val[explainer].append(hfid)
            hfid_dict[explainer] += hfid
            
        elif explainer == "GradCAM":
            values = explanations.node_imp
            max_h_fid = 0
            for i in range(number_of_nodes):
                mask = top_k_percent_nodes(values, i)
                hfid = hfidelity(data, mask, model, target_class, mask_func_node)
                max_h_fid = max(hfid, max_h_fid)
                sparsity =  1 - i/number_of_nodes
                if sparsity < min_sparsity:
                    break
            hfid_val[explainer].append(max_h_fid)
            hfid_dict[explainer] += max_h_fid
        else: 
            pass
for explainer in hfid_dict.keys():
    res_dict[(explainer, run)] = hfid_dict[explainer]/len(explanation_data)
    with open(f"{dataset_name}_res.pkl", "wb") as file:
        pickle.dump(res_dict, file)
# Clear variables and free up memory
#del explain_dict, hfid_dict, hfid_val, explanation_data, data, mask, values, max_h_fid, sparsity, target_class
torch.cuda.empty_cache()
gc.collect()


### T-Test

In [33]:
from scipy.stats import ttest_ind
dataset_name = "BBBP"
#with open(f"{dataset_name}_hfid_results.pkl", "rb") as file:
with open("BBBP_Gstarx_hfid.pkl", "rb") as file:
    t = pickle.load(file)
# Perform t-test between ESPAM and other methods
#with open(f"{dataset_name}_ESPAM_results_{l}.pkl", "rb") as file:
#    espam_values = pickle.load(file)
espam_values = list(map(lambda x: x.cpu().detach().numpy(), hfid_val['ESPAM']))
results = {}
print(hfid_val["ESPAM"])
for method, values in t.items():
    if method != 'ESPAM':
        t_stat, p_value = ttest_ind(espam_values, list(map(lambda x: x.cpu().detach().numpy(),values)))
        results[method] = {'t_stat': t_stat, 'p_value': p_value}

# Print the results
for method, result in results.items():
    print(f"Dataset: {dataset_name} Method: {method}, t-statistic: {result['t_stat']}, p-value: {result['p_value']}")

[tensor(0.5460, grad_fn=<DivBackward0>), tensor(0.5368, grad_fn=<DivBackward0>), tensor(0.5421, grad_fn=<DivBackward0>), tensor(0.5225, grad_fn=<DivBackward0>), tensor(0.5493, grad_fn=<DivBackward0>), tensor(0.5872, grad_fn=<DivBackward0>), tensor(0.5643, grad_fn=<DivBackward0>), tensor(0.5402, grad_fn=<DivBackward0>), tensor(0.5226, grad_fn=<DivBackward0>), tensor(0.5477, grad_fn=<DivBackward0>), tensor(0.5077, grad_fn=<DivBackward0>), tensor(0.6277, grad_fn=<DivBackward0>), tensor(0.5406, grad_fn=<DivBackward0>), tensor(0.5415, grad_fn=<DivBackward0>), tensor(0.5437, grad_fn=<DivBackward0>), tensor(0.5059, grad_fn=<DivBackward0>), tensor(0.5139, grad_fn=<DivBackward0>), tensor(0.5354, grad_fn=<DivBackward0>), tensor(0.5558, grad_fn=<DivBackward0>), tensor(0.5869, grad_fn=<DivBackward0>), tensor(0.5175, grad_fn=<DivBackward0>), tensor(0.5951, grad_fn=<DivBackward0>), tensor(0.5363, grad_fn=<DivBackward0>), tensor(0.5285, grad_fn=<DivBackward0>), tensor(0.6269, grad_fn=<DivBackward0>),